### Deep Learning approach using PyTorch

***Introduction***:
This is a simple Deep Learning approach to Text classification problem. It is very inefficient since it takes a long time to train and purely uses the "text" column. It scores about 78% on the validation set and goes down to a loss of about 0.2 when trained for 100 epochs.


# **Imports**
We need the following packages:
* pandas for loading the dataset
* pytorch building our deep learning model
* numpy for performing some calculations
* tqdm for showing training progress
* nltk for pre-processing (removing stop words)
* re for pre-processing (filtering strings)

In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
import nltk
from nltk.corpus import stopwords
import re

# **Hyperparameters**
We need to define hyperparameters such as the batch size and learnign rate. This is done at the start of the code rather than later for readability.

Here is a list of what they do:
1. BATCH_SIZE : Defines how many items are used each training step
2. LEARNING_RATE : Defines the learning rate aka how much the models weights are updated each training step
3. EPOCHS : For how long the model will train for
4. VAL_SPLIT = This indicates how much data we use for training and validation. If it is set to 0.05, 95% of the dataset from train.csv will be used for training and the other 5% will be used to meassure the accuracy. 

In [2]:
BATCH_SIZE = 64
LEARNING_RATE = 0.01
EPOCHS = 100
VAL_SPLIT = 0.05

# **Downloading list of stop words**
Stop words are words that do not add meaning to a sentence. They are considered obsolete and are often removed prior to training NLP models. 

Here is a sample list of stop words: and, but, how, in, on, or, the, what, will

Example:
**What is a** Neural Network**?**

The code below downloads a list of english stopwords using nltk

In [3]:
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jeffwa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Building the encoder/decoder function**
Next we create the encoder/decoder function. While we don't need a decoder function for text classification, I have included it anyways. 
The encoder function splits the string into all of its words and looks up the index of each one. This way we get a list of integers that can be pased through the embedding layer of the model

In [4]:
vocabulary = []

def encode(str):
    return [vocabulary.index(word) for word in str.split(' ') if word not in stop_words]


def decode(seq):
    return "".join([vocabulary[index] for index in seq])

Our Dataset class holds the train, test or validation dataset. The train and validation dataset can be retrieved by splliting the train.csv as specified in the VAL_SPLIT. 

There are five main functions:
* **collate_fn**: The collate_fn function can be parsed to the DataLoader later. We need this because all items in a batch need to have the same length. All sentences are padded with zeros to the max length using the pad_sequence function provided by PyTorch.
* **len**: Returns the length of the dataset. This is required by PyTorch
* **getitem**: Returns an item from the dataset
* **clean**: This function pre-processes the dataset. At first, it converts the entire dataset to lower case. Why? Because **HeLlO** and **hello** are the same thing. Then we filter out all non-alphabetic characters and whitespace from our dataset, using a [regular expression](https://learn.microsoft.com/en-us/dotnet/standard/base-types/regular-expression-language-quick-reference). Finally, all usernames are stripped since they add little meaning and we apply encoding to the entire DataFrame. 
* **add_vocab**: This adds all words to the vocavulary

In [5]:
class dataset(Dataset):
    def __init__(self, split='train'):
        self.split = split
        
        if split == 'train':
            self.dataset = pd.read_csv('train.csv')
            '''self.dataset = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')'''
            self.dataset = self.dataset.iloc[:round(len(self.dataset) * (1 - VAL_SPLIT))]
            
            self.clean()
            
            self.X = self.dataset['text']
            self.Y = self.dataset['target']
        elif split == 'val':
            self.dataset = pd.read_csv('train.csv')
            '''self.dataset = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')'''
            self.dataset = self.dataset.iloc[round(len(self.dataset) * (1 - VAL_SPLIT)):]
            
            self.clean()
            
            self.X = self.dataset['text']
            self.Y = self.dataset['target']
        else:
            self.dataset = pd.read_csv('test.csv')
            '''self.dataset = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')'''
            self.clean()
            
            self.X = self.dataset['text']
    
    def add_vocab(self):
        for str in self.dataset['text']:
            for item in str.split(' '):
                if item not in vocabulary and item not in stop_words:
                    vocabulary.append(item)
            
    def clean(self):
        self.dataset['text'] = self.dataset['text'].str.lower()
        self.dataset['text'] = self.dataset['text'].str.replace('@\w+', '', regex=True) # remove usernames
        self.dataset['text'] = self.dataset['text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))
        
        self.add_vocab()
        
        self.dataset['text'] = self.dataset['text'].apply(encode)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.split == 'train' or self.split == 'val':
            return torch.tensor(self.X.iloc[idx], dtype=torch.long), torch.tensor(self.Y.iloc[idx], dtype=torch.long)
        else:
            return torch.tensor(self.X.iloc[idx], dtype=torch.long), torch.tensor(self.dataset['id'].iloc[idx], dtype=torch.int)

def collate_fn(batch):
    x, y = zip(*batch)
    return pad_sequence(x, batch_first=True, padding_value=0), torch.tensor(y)


# **Creating the Model**
Our model consists of one [Embedding layer](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) and four [Fully connected layers](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html). As the embedding layer expands the dimensions of our input, we use torch.mean to reduce its dimension.

In [6]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.embedding = nn.Embedding(len(vocabulary), 256) # vocab size x embedding dimension
        self.fc1 = nn.Linear(256, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 2)
        
    def forward(self, x):
        x = self.embedding(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.mean(x, dim=1) 
        x = self.fc4(x)
        
        return x
        

# **Training the model**
First of all we start off by creating two instances of the Dataset class, one for training, one for validation, one for testing. Then we create a DataLoader for each of them.

A new instance of our model, our optimizer and a CrossEntropyLoss function.

We define a function to calculate the accuracy by counting the total amount of correct predictions of a batch. 

We define a function to evaluate the models' performance on the validation dataset using the previously defined function.

Then for each epoch we loop through the batches and calculate the loss between the target batch and the networks predictions. Then we adjust the networks' weights and biases using backpropagation. The models' accuracy on the validation dataset is assesed after each epoch.


In [7]:
train_dataset = dataset(split='train')
trainDataLoader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

val_dataset = dataset(split='val')
valDataLoader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

test_dataset = dataset(split='test')
testDataLoader = DataLoader(test_dataset, batch_size=1, shuffle=False)

net = Classifier()
optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

def calculate_accuracy(predictions, targets):
    predicted_labels = torch.argmax(predictions, dim=1)
    correct_predictions = (predicted_labels == targets).sum().item()
    total_predictions = targets.size(0)
    accuracy = correct_predictions / total_predictions
    return accuracy

def evaluate():
    accuracies = []
    for idx, batch in enumerate(valDataLoader):
        x, y = batch
        
        with torch.no_grad():
            accuracies.append(calculate_accuracy(net.forward(x), y))
        
    print(f'Validation accuracy {np.average(accuracies)}')
    
for epoch in range(EPOCHS):
    progress_bar = tqdm(trainDataLoader)
    
    accuracies = []
    losses = []
    
    for index, (x, y) in enumerate(progress_bar):        
        optimizer.zero_grad()
        
        output = net.forward(x)
        
        loss = criterion(output, y)
        
        
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        accuracies.append(calculate_accuracy(output, y))
        
        progress_bar.set_description(f'Epoch [{epoch+1}/{EPOCHS}]')
        progress_bar.set_postfix(loss=np.average(losses), acc=np.average(accuracies))
        
    evaluate()

Epoch [1/100]: 100%|███| 113/113 [00:03<00:00, 35.54it/s, acc=0.572, loss=0.681]


Validation accuracy 0.45436304644808745


Epoch [2/100]: 100%|███| 113/113 [00:03<00:00, 36.20it/s, acc=0.576, loss=0.676]


Validation accuracy 0.45436304644808745


Epoch [3/100]: 100%|███| 113/113 [00:03<00:00, 34.78it/s, acc=0.576, loss=0.675]


...

Validation accuracy 0.7971738387978142


Epoch [99/100]: 100%|██| 113/113 [00:02<00:00, 41.28it/s, acc=0.887, loss=0.286]


Validation accuracy 0.7902578551912569


Epoch [100/100]: 100%|█| 113/113 [00:02<00:00, 42.73it/s, acc=0.894, loss=0.271]


Validation accuracy 0.7929900956284154


# **Submission**

A new pandas DataFrame is created to hold the data. We then loop through the DataLoader and predict for each tweet. 

In [8]:
import pandas as pd

results = pd.DataFrame({'id': pd.Series(dtype=int), 'target': pd.Series()})

for batch in testDataLoader:
    text, id = batch
    output = net.forward(text)
    
    result_batch = pd.DataFrame({'id': [id.item()], 'target': [torch.argmax(output).item()]})
    results = pd.concat([results, result_batch])

results.to_csv('submission.csv', index=False)

/tmp/ipykernel_19573/3033033919.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  results = pd.DataFrame({'id': pd.Series(dtype=int), 'target': pd.Series()})
